In [57]:
import requests
import psycopg2

The CoinGecko /coins/markets endpoint is publicly accessible (no authentication required for demo use) and returns real-world, market-oriented data

In [58]:
API_URL = "https://api.coingecko.com/api/v3/coins/markets"
PARAMS = {
    "vs_currency": "usd",
    "order": "market_cap_desc",
    "per_page": 100,
    "page": 1,
    "sparkline": False
}

In [ ]:
DB_CONFIG = {
    "host": "<host>",
    "dbname": "ELT_DEMO",
    "user": "<user_name>",
    "password": "<password>",
    "port": 5432
}

Fetch Data - Extract

In [60]:
def fetch_market_data():
    resp = requests.get(API_URL, params=PARAMS)
    resp.raise_for_status()
    return resp.json()

Load data to database - LOAD step

In [61]:
def load_to_postgres(data):
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    cur.execute("""
        CREATE TABLE IF NOT EXISTS staging.raw_markets (
            id TEXT PRIMARY KEY,
            symbol TEXT,
            name TEXT,
            current_price NUMERIC,
            market_cap NUMERIC,
            total_volume NUMERIC,
            high_24h NUMERIC,
            low_24h NUMERIC,
            price_change_24h NUMERIC,
            price_change_percentage_24h NUMERIC,
            circulating_supply NUMERIC,
            total_supply NUMERIC,
            max_supply NUMERIC,
            ath NUMERIC,
            ath_date TIMESTAMPTZ,
            last_updated TIMESTAMPTZ
        )
    """)
    for coin in data:
        ## Insert records while ensuring idempotency -> An operation is idempotent if running it multiple times has the same effect as running it once
        cur.execute("""
            INSERT INTO staging.raw_markets VALUES (
                %(id)s, %(symbol)s, %(name)s, %(current_price)s,
                %(market_cap)s, %(total_volume)s, %(high_24h)s,
                %(low_24h)s, %(price_change_24h)s,
                %(price_change_percentage_24h)s, %(circulating_supply)s,
                %(total_supply)s, %(max_supply)s, %(ath)s,
                %(ath_date)s, %(last_updated)s
            ) ON CONFLICT (id) DO UPDATE SET
              current_price = EXCLUDED.current_price,
              market_cap = EXCLUDED.market_cap,
              total_volume = EXCLUDED.total_volume,
              price_change_24h = EXCLUDED.price_change_24h,
              price_change_percentage_24h = EXCLUDED.price_change_percentage_24h,
              last_updated = EXCLUDED.last_updated
        """, coin)
        
    conn.commit()
    cur.close()
    conn.close()

In [62]:
if __name__ == "__main__":
    data = fetch_market_data()
    load_to_postgres(data)